In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from torch.nn import ModuleList
from torch import nn
import torch.optim as optim

# custom libs
from libs.PretrainedModels import AlexNet_cc, SqueezeNet_cc, InceptionV3_cc
from libs.utils import get_model_name, import_dataset, init_model

In [ ]:
random.seed(1996)
np.random.seed(1996)

## Dataset and dataLoaders

In [ ]:
path_dst = 'dataset'
path_gdrive = ''

# parameters for dataloaders
batch_size=32
num_workers=2
drop_last=True

# default mean and std needed by pretrained models from pytorch
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [ ]:
dst = import_dataset(path_dst=path_dst, 
    train_transform=transforms.Compose([
        transforms.Resize(256),
        # using the same as the test because the trash bin is centered in the image
        transforms.CenterCrop(224), # good for inceptionv3?
        transforms.RandomApply(ModuleList([
            transforms.ColorJitter(brightness=.3, hue=.2),
        ]), p=0.3),

        transforms.RandomApply(ModuleList([
            transforms.Grayscale(num_output_channels=3),
        ]), p=0.2),

        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomPerspective(distortion_scale=0.3, p=0.2),
        transforms.RandomEqualize(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]),
    test_transform=transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224), # good for inceptionv3?
        transforms.AutoAugment(transforms.AutoAugmentPolicy.SVHN),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]), path_gdrive=path_gdrive)

dst.create_data_loader(batch_size=batch_size, num_workers=num_workers, drop_last=drop_last)

dst_inceptionv3 = import_dataset(path_dst=path_dst, 
    train_transform=transforms.Compose([
        transforms.Resize(320),
        # using the same as the test because the trash bin is centered in the image
        transforms.CenterCrop(299), # good for inceptionv3?
        transforms.RandomApply(ModuleList([
            transforms.ColorJitter(brightness=.3, hue=.2),
        ]), p=0.3),

        transforms.RandomApply(ModuleList([
            transforms.Grayscale(num_output_channels=3),
        ]), p=0.2),

        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomPerspective(distortion_scale=0.3, p=0.2),
        transforms.RandomEqualize(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]),
    test_transform=transforms.Compose([
        transforms.Resize(320), 
        transforms.CenterCrop(299), # good for inceptionv3?
        transforms.AutoAugment(transforms.AutoAugmentPolicy.SVHN),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]), path_gdrive=path_gdrive)

dst_inceptionv3.create_data_loader(batch_size=batch_size, num_workers=num_workers, drop_last=drop_last)

## Training

In [ ]:
num_epochs = 50

learning_rates = {
    'alexnet': 0.002,
    'squeezenet': 0.00191,
    'inceptionv3': ''
}

momentum = 0.9
criterion = nn.CrossEntropyLoss()

## AlexNet

In [ ]:
alexnet = init_model(creator=AlexNet_cc(), model_name=get_model_name(model_name='AlexNet', lr=str(learning_rates['alexnet'])), feature_extract=True, use_pretrained=True)
trained_model, history = alexnet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['alexnet'], momentum=momentum, criterion=criterion)

## SqueezeNet

In [ ]:
squeezenet = init_model(creator=SqueezeNet_cc(), model_name=get_model_name(model_name='SqueezeNet', lr=str(learning_rates['squeezenet'])), feature_extract=True, use_pretrained=True)
trained_model, history = squeezenet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet'], momentum=momentum, criterion=criterion)

## InceptionV3

In [ ]:
inception_v3 = init_model(creator=InceptionV3_cc(), model_name=get_model_name(model_name='InceptionV3', lr=str(learning_rates['inceptionv3'])), feature_extract=True, use_pretrained=True)
trained_model, history = inception_v3.do_train(dataset=dst_inceptionv3, num_epochs=num_epochs, lr=learning_rates['inceptionv3'], momentum=momentum, criterion=criterion)